## 1988년 IRIS dataset
- 통계학자 피셔(R.A Fisher)의 붓꽃 연구에 기반한 데이터
- 3개의 종류 iris, 종류별 50갸
- 데이터: 4개의 실수로 구성(sepal length, sepal width, petal length, petal width(cm))
- 문제: 4개의 실수 값에 근거하여 꽃 종류를 맞추는 문제

In [ ]:
import torch
from torch import nn, optim # nn : netural network
from sklearn.datasets import load_iris

### IRIS 데이터를 로딩

In [ ]:
iris = load_iris()

print('Flower names: ', iris.target_names)
print('Feature names: ', iris.feature_names)

Flower names:  ['setosa' 'versicolor' 'virginica']
Feature names:  ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


### 데이터 개수를 확인

In [ ]:
flower_features = iris.data # 150개의 꽃들의 4개의 특징(length, width)이 들어있음
flower_classification = iris.target # 150개의 꽃들을 하나씩 분류해서 들어있음

# print(flower_classification)

print('flower features len: ', len(flower_features))
print('flower classification len: ', len(flower_classification))

flower features len:  150
flower classification len:  150


### 데이터를 실제로 확인

In [ ]:
print(flower_features[0])
print(flower_features[50])
print(flower_features[100])

[5.1 3.5 1.4 0.2]
[7.  3.2 4.7 1.4]
[6.3 3.3 6.  2.5]


### 분류데이터를 실제로 확인. 0,1,2로 꽃 종류를 구분

In [ ]:
print(flower_classification[0])
print(flower_classification[50])
print(flower_classification[100])

0
1
2


## Cross Entropy Loss by Pytorch
- 3가지 종류의 꽃을 맞추기 위해 예측 값 3개 중 가장 높은 숫자 꽃의 종류가 정답일거라고 가정

### prediction [6, 3, 0.1]에 대해서, ground truth와 비교하여 loss 계산

In [ ]:
# 여기서는 첫번째 종류의 꽃의 숫자가 6으로 제일 크기 때문에 정답은 첫번째, 즉 0번째 꽃이다.
import torch
import torch.nn

loss_fn = nn.CrossEntropyLoss()

prediction = torch.tensor([[6.0, 3.0, 0.1]], dtype=torch.float) # 행렬에 무조건 소수점이 들어와야 함
gt = torch.tensor([0])  # 정답은 0번째 꽃

print('prediction size: ', prediction.size())
print('ground truth size: ', gt.size())

prediction size:  torch.Size([1, 3])
ground truth size:  torch.Size([1])


### 정답이 0일 때, Prediction이 이를 맞추었기 때문에 loss값이 작다

In [ ]:
loss_res = loss_fn(prediction, gt)
print('loss for 0: ', loss_res)

loss for 0:  tensor(0.0512)


### 정답이 1이므로 loss값이 크다

In [ ]:
gt = torch.tensor([1])
loss_res = loss_fn(prediction, gt)
print('loss for 1: ', loss_res)

loss for 1:  tensor(3.0512)


### 정답이 2인데, prediction에서는 이의 가능성을 가장 낮게 평가했으므로, loss가 가장 크다.

In [ ]:
gt = torch.tensor([2])
loss_res = loss_fn(prediction, gt)
print('loss for 2: ', loss_res)

loss for 2:  tensor(5.9512)


## IRIS Classification

### 4개의 값을 이용하여 꽃 종류를 맞추는 모델

In [ ]:
net = nn.Linear(4,3)  # 4개를 넣어서 3개로 나온다.
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.25)

### Train과 Test 데이터로 분리

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(flower_features, flower_classification, test_size=0.1)  # test_size=0.1 -> train 90%, test 10%
print(type(x_train), x_train.dtype)

<class 'numpy.ndarray'> float64


### Tensor로 변환

In [ ]:
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.int64) 

In [ ]:
print('x_train: ', x_train.size(), x_train.dtype)
print('y_train: ', y_train.size(), y_train.dtype)

net = net.train()
for i in range(1000):
  optimizer.zero_grad()
  X = net(x_train)
  loss = loss_fn(X, y_train)
  loss.backward()
  optimizer.step()

x_train:  torch.Size([135, 4]) torch.float32
y_train:  torch.Size([135]) torch.int64


### 테스트 과정

In [ ]:
net.eval()  #eval 모델로 시험 -> 시험을 보러 가야한다

with torch.no_grad(): # 시험보는 데이터로 학습을 하면 안되니까 써주는 것?
  prediction_res = net(x_test)
  print('Prediction size: ', prediction_res.size())

  print('Prediction: ', prediction_res)
  print('Ground truth: ', y_test)

Prediction size:  torch.Size([15, 3])
Prediction:  tensor([[ 13.2441,   7.0642, -15.5936],
        [ -7.8542,   3.1702,   9.3028],
        [ -0.9361,   4.2810,   1.2966],
        [ -5.7084,   2.9723,   7.0783],
        [ -1.2820,   4.7156,   0.6961],
        [ -5.3949,   3.4133,   6.5373],
        [  0.2407,   5.7073,  -0.5431],
        [ 11.2859,   6.6912, -13.5635],
        [ -2.4567,   4.5888,   2.5322],
        [ -0.2178,   5.5798,  -0.0509],
        [ 11.0913,   6.6128, -13.3181],
        [ -4.3076,   4.7294,   4.6465],
        [ -1.6413,   5.5774,   1.6267],
        [ 13.4716,   6.9656, -15.8269],
        [ 11.8376,   6.4108, -14.0616]])
Ground truth:  tensor([0, 2, 1, 2, 1, 2, 1, 0, 1, 1, 0, 2, 1, 0, 0])


### 테스트 데이터에 대해서 정답율 체크

In [ ]:
pred = torch.argmax(prediction_res, dim=1)    # 가장 큰 값의 위치를 반환
print(pred)

tensor([0, 2, 1, 2, 1, 2, 1, 0, 1, 1, 0, 1, 1, 0, 0])


In [ ]:
num_correct = (pred == y_test).sum().item()
print('Accuracy: ', num_correct*100 / len(y_test), '%')

Accuracy:  93.33333333333333 %
